# 1.LangChain LCEL 的妙用

- LCEL (LangChain Expression Language) 让链式组合变得简洁优雅，支持并行处理、条件分支、流式输出等强大功能
- Chain：链，用于将多个组件（提示模板、LLM模型、记忆、工具等）连接起来，形成可复用的工作流,完成复杂的任务。Chain 的核心思想是通过组合不同的模块化单元，实现比单一组件更强大的功能。比如：
    - 将 LLM 与 Prompt Template （提示模板）结合
    - 将 LLM 与 输出解析器 结合
    - 将 LLM 与 外部数据 结合，例如用于问答
    - 将 LLM 与 长期记忆 结合，例如用于聊天历史记录
通过将 第一个LLM 的输出作为 第二个LLM 的输入，...，将多个LLM按顺序结合在一起

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os, dotenv

dotenv.load_dotenv()

# 设置API密钥（请替换为你的实际密钥）
chat_model = ChatOpenAI(
    model = "deepseek-chat",
    base_url = os.environ["BASE_URL"],
    api_key = os.environ["DEEPSEEK_API_KEY"]
)

## 1.1. 简洁的链式组合 - 使用 | 操作符

In [4]:
# 传统方式：需要显式调用每个步骤
# prompt = ChatPromptTemplate.from_template("请用一句话总结：{topic}")
# model = ChatOpenAI()
# output_parser = StrOutputParser()
# result = output_parser.parse(model.invoke(prompt.invoke({"topic": "人工智能"})))
# print("result:", result.content)

# LCEL方式：一行代码完成所有步骤
prompt = ChatPromptTemplate.from_template("请用一句话总结：{topic}")
chain = prompt | chat_model | StrOutputParser()

# 使用链
result = chain.invoke({"topic": "人工智能"})
print("result:", result)


result: 人工智能是让机器模拟人类智能以执行复杂任务并实现自主优化的科学技术。


## 1.2. 并行处理多个输入 - batch 调用


In [5]:
# 一次性处理多个输入，自动并行
topics = ["机器学习", "深度学习", "自然语言处理"]
results = chain.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"{topic}: {result}")


机器学习: 机器学习是让计算机通过数据自动学习和改进，而无需显式编程的算法科学。
深度学习: 深度学习是一种通过多层神经网络模拟人脑处理信息机制的机器学习方法，能够自动从数据中学习复杂的特征和模式。
自然语言处理: 好的，这是一句总结：

自然语言处理是让计算机理解、解释和生成人类语言的技术。

如果您需要不同风格的总结，这里还有几个版本：

*   **通俗版：** 教电脑听懂人话、说人话的技术。
*   **核心任务版：** 它致力于在计算机与人之间搭建一座用自然语言沟通的桥梁。
*   **目标版：** 其最终目标是实现人与机器之间无缝的自然语言交流。


## 1.3. 流式输出 - 实时获取结果


In [6]:
# 流式输出，实时显示生成的内容
print("流式输出示例:")
for chunk in chain.stream({"topic": "大语言模型"}):
    print(chunk, end="", flush=True)
print("\n")


流式输出示例:
大语言模型是基于海量数据训练的、能够理解和生成人类语言的深度学习模型。



## 1.4. 复杂链式组合 - 多步骤处理


In [7]:
# 定义多个步骤的链
prompt1 = ChatPromptTemplate.from_template("将以下内容翻译成英文：{text}")
prompt2 = ChatPromptTemplate.from_template("将以下英文内容总结成一句话：{text}")

# 组合多个链：翻译 -> 总结
translate_chain = prompt1 | chat_model | StrOutputParser()
summarize_chain = prompt2 | chat_model | StrOutputParser()

# 正确导入RunnablePassthrough
try:
    from langchain.schema.runnable import RunnablePassthrough
except ImportError:
    from langchain_core.runnables import RunnablePassthrough  # 适用于新版langchain

# 对比: 仅翻译, 以及 翻译后总结
input_text = "在科技浪潮的推动下，人工智能（AI）已从实验室走向日常生活，成为重塑社会形态的核心力量。从工作模式到医疗健康，从教育体系到交通出行，AI正以“润物细无声”的方式渗透至人类生活的每个角落"

# 只做翻译
translated = translate_chain.invoke({"text": input_text})
print("仅翻译结果:", translated)

# 翻译后做总结
summarized = summarize_chain.invoke({"text": translated})
print("翻译后summarize_chain总结结果:", summarized)

# 复合链：自动传递并处理
composite_chain = (
    {"text": RunnablePassthrough()} 
    | translate_chain 
    | (lambda x: {"text": x})  # 将结果转换为字典格式
    | summarize_chain
)
result = composite_chain.invoke({"text": input_text})
print("复合链翻译并总结结果:", result)


仅翻译结果: Driven by the wave of technology, artificial intelligence (AI) has moved from the laboratory into daily life, becoming a core force in reshaping the structure of society. From work patterns to healthcare, and from education systems to transportation, AI is permeating every corner of human life in a subtle yet profound manner.
翻译后summarize_chain总结结果: 人工智能正从实验室走向日常生活，成为重塑社会结构的核心力量，深刻渗透到工作、医疗、教育和交通等各个领域。
复合链翻译并总结结果: 在技术浪潮推动下，人工智能已如春雨般“润物细无声”地渗透到工作、医疗、教育和交通等各个领域，从实验室走向日常生活，成为重塑社会结构的核心力量。


## 1.5. 条件分支 - 根据条件选择不同的处理路径


In [8]:
try:
    from langchain.schema.runnable import RunnableBranch
except ImportError:
    from langchain_core.runnables import RunnableBranch  # 适用于新版 langchain

def is_question(text: str) -> bool:
    return "?" in text or "？" in text   # 根据text是否包含中文问号 或 英文问号,来判断text是否为一个问题

question_chain = ChatPromptTemplate.from_template("请回答这个问题：{text}") | chat_model | StrOutputParser()
statement_chain = ChatPromptTemplate.from_template("请总结这句话：{text}") | chat_model | StrOutputParser()

# 将statement_chain作为默认分支传递 （不要用lambda True作为条件！）
branch_chain = RunnableBranch(
    (lambda x: is_question(x["text"]), question_chain), # True
    statement_chain  # False,默认分支,默认当前表述不是问题
)

# 测试
result1 = branch_chain.invoke({"text": "什么是机器学习？"})
print("问题处理:", result1)

print("=" * 50)

result2 = branch_chain.invoke({"text": "人工智能很强大"})
print("陈述处理:", result2)


问题处理: 当然！这是一个关于“机器学习”的详细且易于理解的解释。

### 一句话概括

**机器学习是人工智能的一个分支，其核心是让计算机通过数据“学习”规律，而不是通过明确的程序指令。**

---

### 详细解释

我们可以把它想象成教一个孩子认动物。

1.  **传统编程方法**：你需要编写一套极其详细的规则，比如“有四条腿、有尾巴、会汪汪叫的是狗”，“有翅膀、会飞、有喙的是鸟”。这种方法非常僵化，如果遇到一只不会飞的鸟（比如鸵鸟），程序可能就无法识别。

2.  **机器学习方法**：你不需要告诉孩子规则。你只需要给他看大量不同动物的图片，并告诉他每张图片是什么（比如“这是狗”、“这是猫”、“这是鸟”）。通过观察成百上千的图片，孩子的大脑会自己总结出狗、猫、鸟的特征（比如狗的耳朵形状、猫的胡须、鸟的翅膀）。最终，当你给他看一张他从未见过的狗的图片时，他也能正确地识别出来。

**在这个比喻中：**
*   **计算机/算法** = 孩子的大脑
*   **数据（图片）** = 大量的动物图片
*   **模型** = 孩子大脑中形成的对每种动物的“概念”或“规律”
*   **训练** = 给孩子看图片并纠正他错误的过程

---

### 机器学习的主要类型

机器学习通常分为三大类：

1.  **监督学习**
    *   **核心思想**：给算法提供“带标签”的数据进行训练。就像有老师在一旁指导。
    *   **例子**：上面的动物识别例子就是监督学习。数据包括“特征”（图片的像素）和“标签”（动物名称）。
    *   **常见任务**：
        *   **分类**：预测离散的类别。例如：垃圾邮件识别（是垃圾邮件/不是垃圾邮件）、图像识别（猫/狗/鸟）。
        *   **回归**：预测连续的数值。例如：预测房价、预测股票价格。

2.  **无监督学习**
    *   **核心思想**：给算法提供“没有标签”的数据，让算法自己发现数据中的内在结构或模式。就像让学生自己从一堆乱序的书中找出分类方法。
    *   **例子**：对电商用户进行分群。算法通过分析用户的购买行为、浏览历史等，自动将用户分成不同的群体（比如“高价值客户”、“折扣追求者”、“新用户”），而无需事先知道这些群体是什么。
    *

## 1.6. 异步支持 - 高效并发处理

In [9]:
import asyncio

# LCEL 天然支持异步
async def async_demo():
    results = await chain.ainvoke({"topic": "量子计算"})
    print("异步调用结果:", results)
    
    # 批量异步调用
    inputs = [{"topic": f"主题{i}"} for i in range(3)]
    results = await chain.abatch(inputs)
    print("批量异步结果:", results)

# Jupyter环境下推荐用如下方式启动异步任务，避免 asyncio.run 报错
await async_demo()


异步调用结果: 量子计算是一种利用量子力学原理（如叠加和纠缠）来存储和处理信息的新型计算范式，它有潜力在特定问题上远超经典计算机。
批量异步结果: ['好的，这是一句总结：\n\n**主题0是文本内容中最核心、最根本的中心思想或主旨。**', '好的，这里有几个不同角度的版本供您选择：\n\n**1. 简洁通用版**\n这句话概括了主题1的核心内容与本质特征。\n\n**2. 强调核心版**\n主题1的精髓在于其核心原则与根本目标。\n\n**3. 点明价值版**\n理解主题1的关键，在于把握其核心概念与重要意义。\n\n---\n\n您能告诉我“主题1”具体指什么吗？例如是“人工智能”、“可持续发展”还是其他？这样我就能为您提供一句真正精准的总结。', '好的，这是一句话总结：\n\n**主题2的核心内容是：通过“问题-原因-解决方案”的结构，将复杂信息清晰、有条理地组织起来，以实现高效沟通。**\n\n（注：由于“主题2”本身是一个占位符，此总结是基于您之前提供的关于“结构化表达”的内容提炼而成。）']


## 1.7. 链式组合的优势总结
- **简洁性**：使用 `|` 操作符，代码更易读
- **可组合性**：轻松组合多个组件
- **并行处理**：自动优化批量调用
- **流式输出**：实时获取结果，提升用户体验
- **异步支持**：天然支持异步操作
- **类型安全**：更好的类型推断和检查
- **调试友好**：可以轻松插入中间步骤进行调试


# 2. 基于LCEL构建的Chains的类型

- 最新的基于LCEL构建的Chains API如下:
    - create_sql_query_chain
    - create_stuff_documents_chain
    - create_openai_fn_runnable
    - create_structured_output_runnable
    - load_query_constructor_runnable
    - create_history_aware_retriever
    - create_retrieval_chain

## 2.1. create_sql_query_chain

- create_sql_query_chain，SQL查询链，是创建生成SQL查询的链，用于将自然语言转换成数据库的SQL查询

In [35]:
from langchain_classic.chains.sql_database.query import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 连接 MySQL 数据库
db_user = "root"
db_password = "123456"
db_host = "127.0.0.1"
db_port = "3306"
db_name = "nacos"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
print("哪种数据库：", db.dialect)
print("获取数据表：", db.get_usable_table_names())
# 执行查询
sql = "SELECT data_id FROM config_info;"
res = db.run(sql)
print(f"查询结果：{res}")

print("=== 调用大模型,根据用户问题生成SQL ===")
chain = create_sql_query_chain(llm=chat_model, db=db)

response = chain.invoke({"question": "nacos服务有多少条配置记录？", "table_names_to_use":["config_info"]})
print(f"生成SQL: {response}")

哪种数据库： mysql
获取数据表： ['config_info', 'config_info_aggr', 'config_info_beta', 'config_info_route', 'config_info_tag', 'config_tags_relation', 'group_capacity', 'his_config_info', 'permissions', 'roles', 'tenant_capacity', 'tenant_info', 'users']
查询结果：[('service.vgroupMapping.admin_tx_group',), ('service.vgroupMapping.crm_tx_group',), ('service.vgroupMapping.examine_tx_group',), ('service.vgroupMapping.hrm_tx_group',), ('service.vgroupMapping.oa_tx_group',), ('store.db.branchTable',), ('store.db.datasource',), ('store.db.dbType',), ('store.db.driverClassName',), ('store.db.globalTable',), ('store.db.lockTable',), ('store.db.maxConn',), ('store.db.maxWait',), ('store.db.minConn',), ('store.db.password',), ('store.db.queryLimit',), ('store.db.url',), ('store.db.user',), ('store.mode',)]
=== 调用大模型,根据用户问题生成SQL ===
生成SQL: SQLQuery: SELECT COUNT(*) AS `config_count` FROM `config_info`


## 2.2. create_stuff_documents_chain

- create_stuff_documents_chain用于将 多个文档内容 合并成 单个长文本 的链式工具，并一次性传递给LLM处理（而不是分多次处理）
- 适合场景:
    - 保持上下文完整，适合需要全局理解所有文档内容的任务（如总结、问答）
    - 适合处理 少量/中等长度文档 的场景

In [36]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 定义提示词模板
prompt = PromptTemplate.from_template("""
如下文档{docs}中说，香蕉是什么颜色的？
""")
# 创建链
llm = ChatOpenAI(model="gpt-4o-mini")
chain = create_stuff_documents_chain(llm, prompt, document_variable_name="docs")
# 文档输入
docs = [
    Document(
        page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一，具有悠久的栽培历史和广泛的分布范围。苹果的原始种"
                 "群主要起源于中亚的天山山脉附近，尤其是现代哈萨克斯坦的阿拉木图地区，提供了所有现代苹果品种的基因库。苹果通过早期的贸易路线，如丝绸之路，从中亚向外扩散到全球各地。"),
    Document(
        page_content="香蕉是白色的水果，主要产自热带地区。"),
    Document(
        page_content="蓝莓是蓝色的浆果，含有抗氧化物质。")
]
# 执行摘要
chain.invoke({"docs": docs})

'根据文中描述，香蕉是白色的水果。'